# Bus Intersection Major Transit Stops, Branching Pathway

In [ ]:
import branching_derived_intersections as bdi
import create_aggregate_stop_frequencies as casf
from update_vars import analysis_date, GCS_FILE_PATH, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD

import pandas as pd
import geopandas as gpd
import numpy as np
from shared_utils import webmap_utils, gtfs_utils_v2, rt_dates, portfolio_utils, rt_utils
import branca

from tqdm import tqdm
tqdm.pandas()
import lookback_wrappers
from segment_speed_utils import helpers
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [ ]:
import importlib
importlib.reload(bdi)

In [ ]:
published_operators_dict = lookback_wrappers.read_published_operators(analysis_date)
trips, lookback_trips_ix = bdi.get_trips_with_route_dir(analysis_date, published_operators_dict)

shapes = helpers.import_scheduled_shapes(analysis_date, columns=["shape_array_key", "geometry"])

shapes = bdi.get_shapes_with_lookback(analysis_date, published_operators_dict, lookback_trips_ix)

max_arrivals_by_stop_single = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop_single_route.parquet")

In [ ]:
singles_explode = bdi.get_explode_singles(max_arrivals_by_stop_single, MS_TRANSIT_THRESHOLD).explode("route_dir")

share_counts = {}
(
    singles_explode.groupby(["schedule_gtfs_dataset_key", "stop_id"]).progress_apply(
        casf.accumulate_share_count, share_counts=share_counts
    )
)

qualify_dict = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= SHARED_STOP_THRESHOLD}
feeds_to_filter = np.unique([key.split("__")[0] for key in qualify_dict.keys()])

hcd_branching_stops = []
unique_qualify_pairs = []
for gtfs_dataset_key in feeds_to_filter:
    this_feed_pairs = bdi.evaluate_overlaps(
        gtfs_dataset_key, show_map=False, shapes=shapes, qualify_dict=qualify_dict
    )
    this_feed_stops = bdi.find_stops_this_feed(gtfs_dataset_key, max_arrivals_by_stop_single, this_feed_pairs)
    hcd_branching_stops += [this_feed_stops]
    unique_qualify_pairs += [this_feed_pairs]
hcd_branching_stops = pd.concat(hcd_branching_stops)
unique_qualify_pairs = pd.concat(unique_qualify_pairs)

In [ ]:
unique_qualify_pairs.route_direction_pair = unique_qualify_pairs.route_direction_pair.astype(str)

In [ ]:
unique_qualify_pairs = portfolio_utils.standardize_operator_info_for_exports(unique_qualify_pairs, analysis_date).drop(columns=['base64_url', 'source_record_id'])

In [ ]:
cmap = branca.colormap.step.Accent_07

In [ ]:
analysis_name_cmap = webmap_utils.categorical_cmap(cmap, unique_qualify_pairs.analysis_name.unique())

In [ ]:
to_map_qualify = unique_qualify_pairs.copy()[unique_qualify_pairs.branching_qualify]
to_map_qualify['color'] = to_map_qualify.analysis_name.map(analysis_name_cmap)

In [ ]:
branching_qualify_state = webmap_utils.set_state_export(to_map_qualify, filename = 'branching_qualify_intersections', cache_seconds=0,
                                                    map_title='Branching Trunk Segments (Intersections)', overwrite=True,
                                                       manual_centroid=[37.336813156889704, -121.88911054161129])

In [ ]:
stops = gcsgp.read_parquet(f"{GCS_FILE_PATH}stops_with_lookback.parquet")

In [ ]:
gdf = stops.merge(hcd_branching_stops, on=['schedule_gtfs_dataset_key', 'stop_id'])
gdf['color'] = [(0, 0, 0)] * len(gdf)

In [ ]:
branching_with_stops_state = webmap_utils.set_state_export(gdf, filename = 'branching_qualify_intersections_stops', cache_seconds=0,
                                                    existing_state = branching_qualify_state, map_title='Branching Trunk Segments (Intersections) w/ Stops', overwrite=True,
                                                       manual_centroid=[37.336813156889704, -121.88911054161129])

## Branching Trunk Segments Maps

In [ ]:
webmap_utils.render_spa_link(branching_qualify_state['spa_link'], text='Trunk Segment Map')

In [ ]:
webmap_utils.render_spa_link(branching_with_stops_state['spa_link'], text= 'Trunk Segment Stop Map')

In [ ]:
webmap_utils.display_spa_map(branching_with_stops_state['spa_link'])

## Full List of Trunk Segments Evaluated

In [ ]:
rt_utils.show_full_df(unique_qualify_pairs)

## Individual Evaluation Maps

In [ ]:
for gtfs_dataset_key in feeds_to_filter:
    this_feed_pairs = bdi.evaluate_overlaps(
        gtfs_dataset_key, show_map=True, shapes=shapes, qualify_dict=qualify_dict
    )